In [2]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [3]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

In [4]:
train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')

In [5]:
counter = Counter()
for line in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter)

In [6]:
# train_iter, val_iter, test_iter = WikiText2()

# i = 0
# for x in test_iter:
#     print(x)
#     i += 1
#     if i > 100:
#         break

# train_data = data_process(train_iter)
# val_data = data_process(val_iter)
# test_data = data_process(test_iter)

In [68]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

def generate_vocabulary(raw_text_iter):
    tokenizer = get_tokenizer('basic_english')
    counter = Counter()
    for x in raw_text_iter:
        counter.update(tokenizer(x))
    return Vocab(counter)

In [70]:
train_iter = WikiText2(root="../data/wikitext", split="train")
vocab = generate_vocabulary(train_iter)

In [71]:
from torch.utils.data import Dataset
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
import torch


class WikitextDataset(Dataset):
    """Wikipedia Language Modelling."""
        
    def __init__(self, root, split, vocab, tokenizer, seq_len=512):
        super().__init__()
        
        self.root = root
        self.split = split
        self.vocab = vocab
        self.seq_len = seq_len
        self.tokenizer = get_tokenizer('basic_english')
        
        data_iter = WikiText2(root=root, split=split)
        self.train_data = self.data_process(data_iter)       
        
    def data_process(self, raw_text_iter):
        data = [torch.tensor([self.vocab[token] for token in self.tokenizer(item)], dtype=torch.long) for item in raw_text_iter]
        data = [x for x in data if x.numel() > 0]
        return torch.cat((data)) 

    def __getitem__(self, index):
        return self.train_data[index*self.seq_len:(index+1)*self.seq_len]

    def __len__(self):
        return len(self.train_data) // self.seq_len

In [72]:
trainset = WikitextDataset(root="../data/wikitext", split="train", vocab=vocab, tokenizer=tokenizer, seq_len=256)
trainset = WikitextDataset(root="../data/wikitext", split="valid", vocab=vocab, tokenizer=tokenizer, seq_len=256)
print(len(trainset))
print(len(trainset.train_data))

837
214417


In [34]:
from torch.utils.data import DataLoader


loader = DataLoader(trainset, batch_size=8)
# print(next(iter(loader)))
print(next(iter(loader)).size())
print(len(loader))

torch.Size([8, 256])
1001


In [2]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric
datasets_list = list_datasets()
len(datasets_list)

886

In [7]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

Reusing dataset wikitext (/home/ash/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)


In [11]:
dataset["test"]

Dataset({
    features: ['text'],
    num_rows: 4358
})

In [25]:
haha = iter(dataset["test"])
print(next(haha))
print(next(haha))
print(next(haha))
print(next(haha))
print(next(haha))
print(next(haha))

{'text': ''}
{'text': ' = Robert Boulter = \n'}
{'text': ''}
{'text': ' Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n'}
{'text': ' In 2006 , Boulter starred alongside Whishaw in the pla

In [30]:
print(dataset.shape)
print(dataset.num_columns)
print(dataset.num_rows)
print(dataset.column_names)

{'test': (4358, 1), 'train': (36718, 1), 'validation': (3760, 1)}
{'test': 1, 'train': 1, 'validation': 1}
{'test': 4358, 'train': 36718, 'validation': 3760}
{'test': ['text'], 'train': ['text'], 'validation': ['text']}


In [32]:
dataset.description

AttributeError: 'DatasetDict' object has no attribute 'description'

In [59]:
# dataset["test"]["text"][:300]

abc = [3, 2 ,5 ,2 ,5, 2, 5]
abc[:50]

[3, 2, 5, 2, 5, 2, 5]